In [2]:
from pandas import DataFrame
import numpy as np
from pulp import *

headings = [184.0, 153.0, 140.0, 122.0, 119.0]
df = DataFrame (headings, columns=['heading'])
df['speed'] = 50
max_speed = 500.0
max_ratio = max_speed / np.min(headings)
df['ratio'] = df.speed/df.heading
conditions_lb = [0,   0.1, 0.2, 0.3, 0.4]
conditions_ub = [0.1, 0.2, 0.3, 0.4, max_speed / np.min(headings)]
choices = [3, 1, 8, 5, 2]
n_range = len(choices)
n_rows = len(df)

# Create primary ratio variables - one for each variable:
df['speed_vars'] = [LpVariable("speed_"+str(j)) for j in range(n_rows)]

# Create auxilary variables - binaries to control
# which bit of range each speed is in
df['aux_vars'] = [[LpVariable("aux_"+str(i)+"_"+str(j), cat='Binary')
                   for i in range(n_range)]
                   for j in range(n_rows)]

# Declare problem
prob = LpProblem("max_pd_column",LpMaximize)

# Define objective function
prob += lpSum([df['aux_vars'][j][i]*choices[i]*headings[j] for i in range(n_range)
               for j in range(n_rows)])

# Constrain only one range to be selected for each row
for j in range(n_rows):
    prob += lpSum([df['aux_vars'][j][i] for i in range(n_range)]) == 1

# Constrain the value of the speed by the ratio range selected
for j in range(n_rows):
    for i in range(n_range):
        prob += df['speed_vars'][j]*(1.0/df['heading'][j]) <= \
                        conditions_ub[i] + (1-df['aux_vars'][j][i])*max_ratio
        prob += df['speed_vars'][j]*(1.0/df['heading'][j]) >= \
                        conditions_lb[i]*df['aux_vars'][j][i]

# Solve problem and print results
prob.solve()

# Dislay the optimums of each var in problem
#for v in prob.variables ():
#    print (v.name, "=", v.varValue)

# Set values in dataframe and print:
df['speed_opt'] = [df['speed_vars'][j].varValue for j in range(n_rows)]
df['ratio_opt'] = df.speed_opt/df.heading
print(df)

   heading  speed     ratio speed_vars  \
0    184.0     50  0.271739    speed_0   
1    153.0     50  0.326797    speed_1   
2    140.0     50  0.357143    speed_2   
3    122.0     50  0.409836    speed_3   
4    119.0     50  0.420168    speed_4   

                                        aux_vars  speed_opt  ratio_opt  
0  [aux_0_0, aux_1_0, aux_2_0, aux_3_0, aux_4_0]       36.8        0.2  
1  [aux_0_1, aux_1_1, aux_2_1, aux_3_1, aux_4_1]       30.6        0.2  
2  [aux_0_2, aux_1_2, aux_2_2, aux_3_2, aux_4_2]       28.0        0.2  
3  [aux_0_3, aux_1_3, aux_2_3, aux_3_3, aux_4_3]       24.4        0.2  
4  [aux_0_4, aux_1_4, aux_2_4, aux_3_4, aux_4_4]       23.8        0.2  
